In [60]:
import pandas as pd
from hazm import (
    Normalizer, word_tokenize, POSTagger,
    Chunker, tree2brackets, Lemmatizer,
    DependencyParser, Stemmer
)

In [61]:
# read data - seprate by ;
df = pd.read_csv('data/verses.csv', sep=';')
print(df.shape)
df.head()

(1384003, 5)


,id,poemId,order,position,text
0,1,2051,1,0,جز نقش تو در نظر نیامد ما را
1,2,2051,2,1,جز کوی تو رهگذر نیامد ما را
2,3,2051,3,0,خواب ارچه خوش آمد همه را در عهدت
3,4,2051,4,1,حقا که به چشم در نیامد ما را
4,5,2052,1,0,بر گیر شراب طرب‌انگیز و بیا


In [62]:
# Initial normalizer, lemmatizer, stemmer
normalizer = Normalizer()
lemmatizer = Lemmatizer()
stemmer = Stemmer()
chunker = Chunker(model='resources/chunker.model')
tagger = POSTagger(model='resources/postagger.model')
parser = DependencyParser(tagger=tagger, lemmatizer=lemmatizer)

In [63]:
verses = df['text'].tolist()

In [64]:
# get structure of first 10 verses
for verse in verses[:10]:
    normalized_verse = normalizer.normalize(verse)
    tokens = word_tokenize(normalized_verse)
    tagged = tagger.tag(tokens)
    # parsed = parser.parse(tagged)
    print("================================")
    print("original_verse: ", verse)
    print("normalized_verse: ", normalized_verse)
    print("tokens: ", tokens)
    print("tagged: ", tagged)
    # print("parsed: ", parsed)
    # print(tree2brackets(chunker.parse(tagged)))

original_verse:  جز نقش تو در نظر نیامد ما را
normalized_verse:  جز نقش تو در نظر نیامد ما را
tokens:  ['جز', 'نقش', 'تو', 'در', 'نظر', 'نیامد', 'ما', 'را']
tagged:  [('جز', 'P'), ('نقش', 'Ne'), ('تو', 'PRO'), ('در', 'P'), ('نظر', 'N'), ('نیامد', 'V'), ('ما', 'PRO'), ('را', 'POSTP')]
original_verse:  جز کوی تو رهگذر نیامد ما را
normalized_verse:  جز کوی تو رهگذر نیامد ما را
tokens:  ['جز', 'کوی', 'تو', 'رهگذر', 'نیامد', 'ما', 'را']
tagged:  [('جز', 'P'), ('کوی', 'Ne'), ('تو', 'PRO'), ('رهگذر', 'N'), ('نیامد', 'V'), ('ما', 'PRO'), ('را', 'POSTP')]
original_verse:  خواب ارچه خوش آمد همه را در عهدت
normalized_verse:  خواب ارچه خوش آمد همه را در عهدت
tokens:  ['خواب', 'ارچه', 'خوش', 'آمد', 'همه', 'را', 'در', 'عهدت']
tagged:  [('خواب', 'N'), ('ارچه', 'ADV'), ('خوش', 'AJ'), ('آمد', 'V'), ('همه', 'PRO'), ('را', 'POSTP'), ('در', 'P'), ('عهدت', 'N')]
original_verse:  حقا که به چشم در نیامد ما را
normalized_verse:  حقا که به چشم در نیامد ما را
tokens:  ['حقا', 'که', 'به', 'چشم', 'در', 'نیامد', '

In [68]:
categories = {}
# categorize tokens
for verse in verses:
    normalized_verse = normalizer.normalize(verse)
    tokens = word_tokenize(normalized_verse)
    tagged = tagger.tag(tokens)
    # add to categories
    for token in tagged:
        print(token)
        if token[1] not in categories:
            categories[token[1]] = [token[0]]
        else:
            categories.get(token[1]).append(token[0])

print(len(categories))


AttributeError: 'float' object has no attribute 'translate'